# Image Preprocessing

In [1]:
# Importing Libraries

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Image Augmentation

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='maC1LOe-mOwumH43U0EIj2H0hqiYqgplSU_RRMU7cjpx',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-127siqa7ts6stp'
object_key = 'Datatest.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
pwd

'/home/wsuser/work'

In [6]:
ls

Datatest/


In [7]:
import os
filenames=os.listdir('/home/wsuser/work/Datatest')
print(filenames)

['test_set', 'training_set']


In [8]:
# Loading train and test set

X_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Datatest/training_set", target_size = (64, 64), batch_size = 32, class_mode = 'categorical')
X_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Datatest/test_set", target_size = (64, 64), batch_size = 32, class_mode = 'categorical')

Found 15130 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [9]:
# checking indices

X_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

# Model Building

In [10]:
# Importing Libraries

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten

In [11]:
# Initializing the Model

model = Sequential()

In [12]:
# Adding Convolution Layer

model.add(Convolution2D((32), (3,3), input_shape = (64, 64, 3), activation = 'relu'))

In [13]:
# Adding Pooling Layer

model.add(MaxPooling2D(pool_size = (2, 2)))

In [14]:
# Adding Flatten Layer

model.add(Flatten())

In [15]:
# Adding Hidden Layer

model.add(Dense(units = 512, kernel_initializer = 'random_uniform', activation = 'relu'))

In [16]:
# Adding Output Layer

model.add(Dense(units = 9, kernel_initializer = 'random_uniform', activation = 'softmax'))

In [17]:
# Compile the model

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [18]:
# Fiiting the model

model.fit_generator(X_train, steps_per_epoch = 24, epochs = 10, validation_data = X_test, validation_steps = 40)

/tmp/wsuser/ipykernel_209/1270027362.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(X_train, steps_per_epoch = 24, epochs = 10, validation_data = X_test, validation_steps = 40)


Epoch 1/10
24/24 [==============================] - 8s 332ms/step - loss: 1.5926 - accuracy: 0.5143 - val_loss: 0.6831 - val_accuracy: 0.7680
Epoch 2/10
24/24 [==============================] - 8s 325ms/step - loss: 0.5332 - accuracy: 0.8346 - val_loss: 0.3981 - val_accuracy: 0.9102
Epoch 3/10
24/24 [==============================] - 8s 350ms/step - loss: 0.3379 - accuracy: 0.9049 - val_loss: 0.4388 - val_accuracy: 0.9180
Epoch 4/10
24/24 [==============================] - 8s 327ms/step - loss: 0.2557 - accuracy: 0.9336 - val_loss: 0.3540 - val_accuracy: 0.9086
Epoch 5/10
24/24 [==============================] - 8s 349ms/step - loss: 0.1962 - accuracy: 0.9362 - val_loss: 0.2927 - val_accuracy: 0.9242
Epoch 6/10
24/24 [==============================] - 8s 324ms/step - loss: 0.1645 - accuracy: 0.9544 - val_loss: 0.2752 - val_accuracy: 0.9227
Epoch 7/10
24/24 [==============================] - 8s 334ms/step - loss: 0.1626 - accuracy: 0.9466 - val_loss: 0.2436 - val_accuracy: 0.9406
Epoch 

In [19]:
# Saving the model

model.save('aslpng1.h5')

In [20]:
!tar -zcvf image-classification-model_new.tgz aslpng1.h5

aslpng1.h5


In [21]:
ls

aslpng1.h5  Datatest/  image-classification-model_new.tgz


In [22]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 16.6 MB/s eta 0:00:01


In [23]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"T01wm0ro66URSrt1Lx7Eufw9grIdgkHvj8aWGgiRFNFT"
}
client = APIClient(wml_credentials)

In [24]:
client = APIClient(wml_credentials)

In [25]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for  item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [26]:
space_uid = guid_from_space_name(client, 'realtimecommunication')
print("space= "+ space_uid)

space= 5f555a1d-4262-431b-9629-c5064f2bb53e


In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list(1000)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [29]:
software_spec_uid = client.software_specifications.get_uid_by_name('runtime-22.2-py3.10-xc')
software_spec_uid

'5e8cddff-db4a-5a6a-b8aa-2d4af9864dab'

In [30]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz' ,meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
)

model_id = client.repository.get_model_uid(model_details)

Failure during creating model. (POST https://us-south.ml.cloud.ibm.com/ml/v4/models?version=2021-06-24&space_id=5f555a1d-4262-431b-9629-c5064f2bb53e)
Status code: 400, body: {
  "trace": "b594792537bb10d898e72314562f0633",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported model type and software specification combination: 'keras_2.2.4' and '5e8cddff-db4a-5a6a-b8aa-2d4af9864dab'",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}


ApiRequestFailure: Failure during creating model. (POST https://us-south.ml.cloud.ibm.com/ml/v4/models?version=2021-06-24&space_id=5f555a1d-4262-431b-9629-c5064f2bb53e)
Status code: 400, body: {
  "trace": "b594792537bb10d898e72314562f0633",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported model type and software specification combination: 'keras_2.2.4' and '5e8cddff-db4a-5a6a-b8aa-2d4af9864dab'",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}

In [31]:
from keras.models import load_model
import numpy as np
import cv2

In [32]:
model=load_model('aslpng1.h5')

In [35]:
from skimage.transform import resize
def detect(frame):
  img = resize(frame,(64,64,1))
  img = np.expand_dims(img,axis=0)
  if(np.max(img)>1):
    img = img/255.0
  prediction = model.predict(img)
  print(prediction)
  prediction = np.argmax(prediction,axis=1)
  print(prediction)